In [2]:
import pandas as pd
import json
import requests
from config import api_key
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from flask import Flask, jsonify, render_template

In [30]:
def get_businesses(location, term, api_key):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break

    return data

In [31]:
#location can be entered using city name or zipcode
location_list = ["Atherton", "Belmont", "Brisbane", "Burlingame", "Colma", "Daly City", "East Palo Alto", "Foster City", 
                 "Half Moon Bay", "Menlo Park", "Millbrae", "Pacifica", "Portola Valley", "Redwood City", "San Bruno", 
                 "San Carlos", "San Francisco", "San Mateo", "South San Francisco", "Woodside", "Campbell", "Cupertino", 
                 "Gilroy", "Los Altos", "Milpitas", "Monte Sereno", "Morgan Hill", "Mountain View", "Palo Alto", "San Jose", 
                 "Santa Clara", "Saratoga", "Sunnyvale"]

term = "restaurant"

business_name = []
business_review_count = []
avg_rating = []
lat = []
long = []
address = []
city_list = []
zipcode = []
phone = []
category = []

for city in location_list:
    response = get_businesses(city, term, api_key)
    #print(response)

    for result in response:
        try:
            business_name.append(result["name"])
            business_review_count.append(result["review_count"])
            avg_rating.append(result["rating"])
            lat.append(result["coordinates"]["latitude"])
            long.append(result["coordinates"]["longitude"])
            address.append(result["location"]["display_address"][0])
            city_list.append(result["location"]["city"])
            zipcode.append(result["location"]["zip_code"])
            phone.append(result["display_phone"])
            category.append(result["categories"][0]["title"])
        except:
            print("No category found")
            category.append(None)

No category found
No category found
No category found


In [32]:
result_df = pd.DataFrame({ "Business_Name" : business_name,
                           "Review_Count" : business_review_count,
                           "Average_Rating" : avg_rating,
                           "Category" : category,
                           "Latitude" : lat,
                           "Longtitude" : long,
                           "Address" : address,
                           "City" : city_list,
                           "Zip_Code" : zipcode,
                           "Phone_Number" : phone })
result_df = result_df.astype({"Business_Name": str, "Review_Count": int, "Average_Rating": float,
                              "Category": str, "Latitude": float, "Longtitude": float, 
                              "Address": str, "City": str, "Zip_Code": str, "Phone_Number": str})
result_df

,Business_Name,Review_Count,Average_Rating,Category,Latitude,Longtitude,Address,City,Zip_Code,Phone_Number
0,ROMA,10,4.5,Italian,37.450925,-122.185503,820 Santa Cruz Ave,Menlo Park,94025,(650) 323-3665
1,Port of Perri Perri,9,4.5,Halal,37.427067,-122.144459,340 California Ave,Palo Alto,94306,(650) 382-2239
2,The Refuge,801,4.0,Sandwiches,37.451286,-122.185944,1143 Crane St,Menlo Park,94025,(650) 319-8197
3,Back A Yard Caribbean American Grill,2101,4.5,American (Traditional),37.472900,-122.154948,1189 Willow Rd,Menlo Park,94025,(650) 323-4244
4,Black Pepper Restaurant,521,4.0,Asian Fusion,37.452880,-122.182220,1029 El Camino Real,Menlo Park,94025,(650) 485-2345
...,...,...,...,...,...,...,...,...,...,...
23409,Round Table Pizza,117,2.0,Pizza,37.338889,-122.042452,1663 Hollenbeck Ave,Sunnyvale,94087,(408) 245-5905
23410,Burger King,77,2.0,Burgers,37.351860,-121.999770,3750 El Camino Real,Santa Clara,95051,(408) 241-6564
23411,Truya Sushi,245,3.5,Sushi Bars,37.352740,-121.990260,3431 El Camino Real,Santa Clara,95051,(408) 244-4534
23412,Applebee's Grill + Bar,607,2.5,American (Traditional),37.316722,-121.975951,555 Saratoga Ave,San Jose,95129,(408) 446-8370


In [33]:
result_df["City"].value_counts()

San Jose           2949
San Francisco      2781
Palo Alto          1670
Redwood City       1588
Santa Clara        1418
                   ... 
Saint Lucia           1
CA                    1
SF                    1
Norman Park           1
Emporium Valley       1
Name: City, Length: 131, dtype: int64

In [34]:
engine = create_engine("sqlite:///sf_restaurant_db.sqlite")
conn = engine.connect()
Base = declarative_base()

In [35]:
class Restaurant(Base):
    __tablename__ = "restaurant_info"
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key = True)
    Business_Name = Column(String(50))
    Review_Count = Column(Integer)
    Average_Rating = Column(Float)
    Category = Column(String(30))
    Latitude = Column(Float)
    Longtitude = Column(Float)
    Address = Column(String(200))
    City = Column(String(10))
    Zip_Code = Column(String(10))
    Phone_Number = Column(String(30))

In [36]:
Base.metadata.create_all(conn)

In [37]:
result_df.to_sql("restaurant_info", con=engine, if_exists='replace', index_label='id')

In [38]:
restaurant_data = pd.read_sql_query('select * from restaurant_info', con=engine)
restaurant_data

,id,Business_Name,Review_Count,Average_Rating,Category,Latitude,Longtitude,Address,City,Zip_Code,Phone_Number
0,0,ROMA,10,4.5,Italian,37.450925,-122.185503,820 Santa Cruz Ave,Menlo Park,94025,(650) 323-3665
1,1,Port of Perri Perri,9,4.5,Halal,37.427067,-122.144459,340 California Ave,Palo Alto,94306,(650) 382-2239
2,2,The Refuge,801,4.0,Sandwiches,37.451286,-122.185944,1143 Crane St,Menlo Park,94025,(650) 319-8197
3,3,Back A Yard Caribbean American Grill,2101,4.5,American (Traditional),37.472900,-122.154948,1189 Willow Rd,Menlo Park,94025,(650) 323-4244
4,4,Black Pepper Restaurant,521,4.0,Asian Fusion,37.452880,-122.182220,1029 El Camino Real,Menlo Park,94025,(650) 485-2345
...,...,...,...,...,...,...,...,...,...,...,...
23409,23409,Round Table Pizza,117,2.0,Pizza,37.338889,-122.042452,1663 Hollenbeck Ave,Sunnyvale,94087,(408) 245-5905
23410,23410,Burger King,77,2.0,Burgers,37.351860,-121.999770,3750 El Camino Real,Santa Clara,95051,(408) 241-6564
23411,23411,Truya Sushi,245,3.5,Sushi Bars,37.352740,-121.990260,3431 El Camino Real,Santa Clara,95051,(408) 244-4534
23412,23412,Applebee's Grill + Bar,607,2.5,American (Traditional),37.316722,-121.975951,555 Saratoga Ave,San Jose,95129,(408) 446-8370


In [39]:
session = Session(bind=engine) 
best_restaurant = session.query(Restaurant).filter(Restaurant.Average_Rating >= 4).filter(Restaurant.Review_Count > 2000).all()

In [40]:
for item in best_restaurant:
    print(item.Business_Name)

Back A Yard Caribbean American Grill
Old Port Lobster Shack
Evvia Estiatorio
Coconuts Caribbean Restaurant & Bar
Oren's Hummus
New England Lobster Market & Eatery
Mingalaba
Ramen Dojo
Millbrae Pancake House
Sushi Sam's Edomata
Little Lucca Specialty Sandwich Shop
Espetus Churrascaria San Mateo
San Tung
Little Lucca Specialty Sandwich Shop
Farmhouse Kitchen Thai Cuisine
El Farolito
Hard Knox Cafe
The Front Porch
Thanh Long
Chez Maman East
La Taqueria
Limón Rotisserie
Izakaya Sozai
Pancho Villa Taqueria
The Monk's Kettle
Millbrae Pancake House
Loló
Tartine Bakery & Cafe
Delfina
Plow
Taqueria Cancún
Foreign Cinema
Little Star Pizza
Pizzeria Delfina - Mission
Beretta
Kitchen Story
Gracias Madre
Back A Yard Caribbean American Grill
Evvia Estiatorio
Coconuts Caribbean Restaurant & Bar
Oren's Hummus
Barbara's Fishtrap
Back A Yard Caribbean American Grill
Old Port Lobster Shack
Evvia Estiatorio
Coconuts Caribbean Restaurant & Bar
Oren's Hummus
The Refuge
TOWN San Carlos
Sushi Tomi
New England 

In [59]:
engine = create_engine("sqlite:///sf_restaurant_db.sqlite")

In [60]:
conn = engine.connect()

In [61]:
data = pd.read_sql_query('select * from restaurant_info', con=engine)
data

,id,Business_Name,Review_Count,Average_Rating,Category,Latitude,Longtitude,Address,City,Zip_Code,Phone_Number
0,0,ROMA,10,4.5,Italian,37.450925,-122.185503,820 Santa Cruz Ave,Menlo Park,94025,(650) 323-3665
1,1,Port of Perri Perri,9,4.5,Halal,37.427067,-122.144459,340 California Ave,Palo Alto,94306,(650) 382-2239
2,2,The Refuge,801,4.0,Sandwiches,37.451286,-122.185944,1143 Crane St,Menlo Park,94025,(650) 319-8197
3,3,Back A Yard Caribbean American Grill,2101,4.5,American (Traditional),37.472900,-122.154948,1189 Willow Rd,Menlo Park,94025,(650) 323-4244
4,4,Black Pepper Restaurant,521,4.0,Asian Fusion,37.452880,-122.182220,1029 El Camino Real,Menlo Park,94025,(650) 485-2345
...,...,...,...,...,...,...,...,...,...,...,...
23409,23409,Round Table Pizza,117,2.0,Pizza,37.338889,-122.042452,1663 Hollenbeck Ave,Sunnyvale,94087,(408) 245-5905
23410,23410,Burger King,77,2.0,Burgers,37.351860,-121.999770,3750 El Camino Real,Santa Clara,95051,(408) 241-6564
23411,23411,Truya Sushi,245,3.5,Sushi Bars,37.352740,-121.990260,3431 El Camino Real,Santa Clara,95051,(408) 244-4534
23412,23412,Applebee's Grill + Bar,607,2.5,American (Traditional),37.316722,-121.975951,555 Saratoga Ave,San Jose,95129,(408) 446-8370


In [62]:
data.drop_duplicates(subset=["Latitude", "Longtitude"], inplace = True) 

In [72]:
data.set_index("id", inplace=True)

In [ ]:
data

In [69]:
engine = create_engine("sqlite:///sf_restaurant_db3.sqlite")
conn = engine.connect()
Base = declarative_base()

In [70]:
class Restaurant(Base):
    __tablename__ = "restaurant_info"
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key = True)
    Business_Name = Column(String(50))
    Review_Count = Column(Integer)
    Average_Rating = Column(Float)
    Category = Column(String(30))
    Latitude = Column(Float)
    Longtitude = Column(Float)
    Address = Column(String(200))
    City = Column(String(10))
    Zip_Code = Column(String(10))
    Phone_Number = Column(String(30))

In [71]:
Base.metadata.create_all(conn)

In [66]:
data.to_sql("restaurant_info", con=engine, if_exists='replace', index_label='id', index=False)

In [67]:
restaurant_data2 = pd.read_sql_query('select * from restaurant_info', con=engine)
restaurant_data2

,id,Business_Name,Review_Count,Average_Rating,Category,Latitude,Longtitude,Address,City,Zip_Code,Phone_Number
0,0,ROMA,10,4.5,Italian,37.450925,-122.185503,820 Santa Cruz Ave,Menlo Park,94025,(650) 323-3665
1,1,Port of Perri Perri,9,4.5,Halal,37.427067,-122.144459,340 California Ave,Palo Alto,94306,(650) 382-2239
2,2,The Refuge,801,4.0,Sandwiches,37.451286,-122.185944,1143 Crane St,Menlo Park,94025,(650) 319-8197
3,3,Back A Yard Caribbean American Grill,2101,4.5,American (Traditional),37.472900,-122.154948,1189 Willow Rd,Menlo Park,94025,(650) 323-4244
4,4,Black Pepper Restaurant,521,4.0,Asian Fusion,37.452880,-122.182220,1029 El Camino Real,Menlo Park,94025,(650) 485-2345
...,...,...,...,...,...,...,...,...,...,...,...
8021,23318,Subway,23,2.5,Sandwiches,37.381171,-122.007802,927 E Arques Ave,Sunnyvale,94085,(408) 732-3436
8022,23343,McDonald's,96,2.0,Fast Food,37.362710,-122.027110,556 E El Camino Real,Sunnyvale,94086,(408) 736-5091
8023,23358,Thanh Restaurant,8,2.0,Vietnamese,37.371864,-121.989487,1294 Kifer Rd,Sunnyvale,94086,(408) 245-2063
8024,23371,McDonald's,18,2.0,Fast Food,37.393440,-122.028970,696 N Mathilda Ave,Sunnyvale,94085,(408) 830-9970
